In [42]:
#Libraries
import random

In [43]:
#Initialise the dictionary
with open("dict.txt") as f:
    dictionary = [el[:-1] for el in f.readlines()]

In [44]:
#Binary Search to check if an element exists


def binarySearch(arr, check):
    l = -1
    r = len(arr)
    while(r-l > 1):
        mid = (l + r)//2
        if check(arr[mid]):
            r = mid
        else:
            l = mid
    return l

def isValid(word):
    l = binarySearch(dictionary, lambda x: x > word)
    if dictionary[l] == word:
        return True
    else:
        return False

In [45]:
#~25-30 secs for 3000 words ~ 10^7 ops
totCharades = []

for a in dictionary[:3000]:
    for b in dictionary[:3000]:
        word = a + b
        if isValid(word):
            totCharades.append((a, b, word))

KeyboardInterrupt: 

In [46]:
#to implement validity checking via trie, we need to add a termination character $

class Trie(object):
    #Implementation of a simple trie

    def  __init__(self, strings):
        self.root = {}
        self.terminator = '$'
        for string in strings: self.add(string)
    
    def add(self, string):
        string += self.terminator
        curr = self.root
        for c in string:
            if c not in curr:
                curr[c] = {}
            curr = curr[c]

    #Get a random string with given prefix, or report none exists:
    def query(self, pref):
        curr = self.root
        result = pref
        for c in pref:
            if c not in curr:
                return None
            curr = curr[c]
        while curr:
            c = random.choice(list(curr.keys()))
            result += c
            curr = curr[c]
        return result
    
    #Check if a word exists
    def isValid(self, word):
        word += self.terminator
        curr = self.root
        for c in word:
            if c not in curr:
                return False
            curr = curr[c]
        return True

fullTrie = Trie(dictionary)

In [47]:
#~4 seconds for 3000 words ~ 10^7 ops - impressive speedup!
#50 secs for 10^8
totCharades = []

for a in dictionary:
    for b in dictionary:
        word = a + b
        if fullTrie.isValid(word):
            totCharades.append((a, b, word))

KeyboardInterrupt: 

# $\mathcal O(nm)$ Charade Generation
Since $m << n$ for a decent dictionary, this presents a polynomial speedup to the $\mathcal O(n^2m)$ algorithm above.

The key idea is that instead of going through each combination of two words, we simply look at each word and see if it can be broken down into two valid words. 

Let $p_i$ represent the prefix upto $i$ characters, ie `word[:i]` and $s_i$ represent the suffix of length $i$ or `word[-i:]`. Then a word of length $m$ is a charade iff $\exists i\in[1,m-1] \ni p_i, s_{m-i}$ are both valid.

In order to efficiently check this, define operation `prefixValidity(word)` on the Trie that returns a boolean array denoting whether each prefix of the array is valid or not. This is done in $\mathcal O(m)$ time and we need to do it twice for each word, giving an overall complexity of $\mathcal O(nm)$

In [48]:
#to implement validity checking via trie, we need to add a termination character $

class Trie(object):
    #Implementation of a simple trie

    def  __init__(self, strings):
        self.root = {}
        self.terminator = '$'
        for string in strings: self.add(string)
    
    def add(self, string):
        string += self.terminator
        curr = self.root
        for c in string:
            if c not in curr:
                curr[c] = {}
            curr = curr[c]

    #Get a random string with given prefix, or report none exists:
    def query(self, pref):
        curr = self.root
        result = pref
        for c in pref:
            if c not in curr:
                return None
            curr = curr[c]
        while curr:
            c = random.choice(list(curr.keys()))
            result += c
            curr = curr[c]
        return result
    
    #Check if a word exists
    def isValid(self, word):
        word += self.terminator
        curr = self.root
        for c in word:
            if c not in curr:
                return False
            curr = curr[c]
        return True
    
    #Return a boolean array denoting the validity of each prefix of the word
    def prefixValidity(self, word):
        result = [False]*(len(word)-1)
        curr = self.root
        for i in range(len(word)-1):
            c = word[i]
            if c not in curr:
                return result
            curr = curr[c]
            if self.terminator in curr:
                result[i] = True
        return result

fullTrie = Trie(dictionary)
suffixTrie = Trie([el[::-1] for el in dictionary])

In [64]:
def findCharadesInWord(word):
    result = []
    p = fullTrie.prefixValidity(word)
    s = suffixTrie.prefixValidity(word[::-1])[::-1]
    for i in range(1, len(word)):
        if p[i-1] and s[i-1]:
            #stricter conditions to prevent lower quality charades
            if i > 2 and len(word)-i > 2:
                result.append((word[:i], word[i:], word))
    return result
    

In [65]:
#3000 words in 0.6s, major speedup
#Gets through the whole dictionary in less than a second

#find all possible charades
def findAllCharades():
    result = []
    for word in dictionary:
        for el in findCharadesInWord(word):
            result.append(el)
    return result

totCharades = findAllCharades()

In [66]:
def generateCharades(n):
    result = list(random.sample(totCharades, n))
    return result

In [123]:
for el in generateCharades(5):
    print(el)

('song', 'stress', 'songstress')
('asp', 'halt', 'asphalt')
('out', 'gun', 'outgun')
('over', 'growth', 'overgrowth')
('over', 'draft', 'overdraft')


Next part kinda optional, it fully generates the charade clue by substituting in synonyms for each word. Uses `wordnet` library

In [74]:
from nltk.corpus import wordnet as wn
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/abhi/nltk_data...


True

In [85]:
def getSynonyms(word):
    result = []
    for el in list(wn.synsets(word)):
        syn = el.name().split('.')[0]
        syn = (' ').join(syn.split('_'))
        if syn != word:
            result.append(syn)
    return result

In [98]:
def formatCharade(charade):
    result = []
    for word in charade:
        syns = getSynonyms(word)
        if len(syns) != 0:
            result.append(random.choice(syns))
        else:
            result.append(word)
    result = result[0] + " + " + result[1] + " = " + result[2]
    return {
        'question': result,
        'answer': charade
    }

In [129]:
for charade in generateCharades(5):
    print(formatCharade(charade)['question'], formatCharade(charade)['answer'], sep="  |  ")

shipshape + ester = spare  |  ('trim', 'ester', 'trimester')
buttocks + about = backmost  |  ('rear', 'most', 'rearmost')
thinly + baron = think  |  ('thin', 'king', 'thinking')
shoot + peg = pippin  |  ('pip', 'pin', 'pippin')
hydraulic + ally = hydraulically  |  ('hydraulic', 'ally', 'hydraulically')
